In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from concurrent.futures import ThreadPoolExecutor, as_completed
from joblib import Parallel, delayed
import xarray as xr
import time
import os
import pygmt

import AbFab as af

%load_ext autoreload
%autoreload 2


In [ ]:
# Example of input parameters (these would be derived from your grids)
params = {
    'H': 50,       # Base rms height in meters
    'kn': 0.2,   # Characteristic width in km^-1 (normal to ridge)
    'ks': 0.02,   # Characteristic length in km^-1 (parallel to ridge)
    'D': 2.2       # Fractal dimension
}

# Example grids (replace with real data)
nx, ny = 512, 512
#seafloor_age = np.random.rand(nx, ny) * 100  # Example seafloor age in Myr
seafloor_age,sediment_thickness = np.meshgrid(np.arange(0,512,1),np.arange(0,512,1))
#sediment_thickness = np.random.rand(nx, ny) * 500  # Example sediment thickness in meters
seafloor_age = seafloor_age+(sediment_thickness*2)
# Generate the synthetic bathymetry
synthetic_bathymetry = af.generate_synthetic_bathymetry_fft((nx, ny), 
                                                         seafloor_age, 
                                                         sediment_thickness*2, 
                                                         params)

# Optionally, plot the bathymetry
plt.imshow(synthetic_bathymetry, cmap='terrain')
plt.colorbar(label="Depth (m)")
plt.title("Synthetic Abyssal Hill Bathymetry with Sediment and Azimuth")
plt.show()

In [ ]:
# Example of input parameters (these would be derived from your grids)
params = {
    'H': 50,       # Base rms height in meters
    'kn': 0.05,   # Characteristic width in km^-1 (normal to ridge)
    'ks': 0.5,   # Characteristic length in km^-1 (parallel to ridge)
    'D': 2.2       # Fractal dimension
}

# Example grids (replace with real data)
nx, ny = 128, 128
#seafloor_age = np.random.rand(nx, ny) * 100  # Example seafloor age in Myr
sediment_thickness = np.random.rand(nx, ny) * 5  # Example sediment thickness in meters

seafloor_age,tmp = np.meshgrid(np.arange(0,ny,1),np.arange(0,nx,1))
#seafloor_age = seafloor_age+(sediment_thickness*2)
seafloor_age = seafloor_age+(tmp*2)


# Generate the synthetic bathymetry
synthetic_bathymetry = af.generate_bathymetry_spatial_filter(seafloor_age, 
                                                              sediment_thickness, 
                                                              params)


fig,ax = plt.subplots(ncols=2)
ax[0].imshow(seafloor_age)
ax[1].imshow(synthetic_bathymetry, cmap='terrain')
#plt.colorbar(label="Depth (m)")
#plt.title("Synthetic Abyssal Hill Bathymetry (Spatial Filter Method)")
plt.show()


In [ ]:
os.cpu_count()

agegrid_filename = '/Users/simon/Data/AgeGrids/2020/age.2020.1.GeeK2007.6m.nc'
sediment_thickness_filename = '/Users/simon/GIT/pyBacktrack/pybacktrack/bundle_data/sediment_thickness/GlobSed.nc'

age_da = xr.open_dataarray(agegrid_filename)
sed_da = pygmt.grdsample(sediment_thickness_filename, region=agegrid_filename)/1000.
minimum_sediment_thickness = 0.1
sed_da.data = sed_da.where(sed_da>=minimum_sediment_thickness, 
                           minimum_sediment_thickness).fillna(minimum_sediment_thickness)

rand_da = age_da.copy()
rand_da.data = af.generate_random_field(rand_da.data.shape)


ny,nx = seafloor_age.shape

coord = (600,1400)
chunksize = 128
agegrid_chunk = age_da.data[coord[0]:coord[0]+chunksize, coord[1]:coord[1]+chunksize]
#sediment_thickness_chunk = np.random.rand(chunksize, chunksize) * 5  # Example sediment thickness in meters
sediment_thickness_chunk = sed_da.data[coord[0]:coord[0]+chunksize, coord[1]:coord[1]+chunksize]
random_chunk = rand_da.data[coord[0]:coord[0]+chunksize, coord[1]:coord[1]+chunksize]

start = time.time()


#TODO check units for sediment thickness

synthetic_bathymetry = af.generate_bathymetry_spatial_filter(agegrid_chunk, 
                                                             sediment_thickness_chunk*10., 
                                                             params,
                                                             random_field=random_chunk)

print(time.time() - start)

fig,ax = plt.subplots(ncols=3)
ax[0].imshow(agegrid_chunk.data)
ax[1].imshow(sediment_thickness_chunk.data)
ax[2].imshow(synthetic_bathymetry, cmap='terrain')
#ax[2].imshow(synthetic_bathymetry2, cmap='terrain')
#plt.colorbar(label="Depth (m)")
#plt.title("Synthetic Abyssal Hill Bathymetry (Spatial Filter Method)")
plt.show()


In [ ]:
import pygmt
age_da = xr.open_dataarray('/Users/simon/Data/AgeGrids/2020/age.2020.1.GeeK2007.6m.nc')

print(age_da.shape)
#age_da.plot()


sed_da = pygmt.grdsample('/Users/simon/GIT/pyBacktrack/pybacktrack/bundle_data/sediment_thickness/GlobSed.nc',
                         region='/Users/simon/Data/AgeGrids/2020/age.2020.1.GeeK2007.6m.nc')

sed_da

rand_da = age_da.copy()
rand_da.data = af.generate_random_field(rand_da.data.shape)

#rand_da.plot()

In [ ]:

#nx = 360
#ny = 200
#x0 = 0
#y0 = 0
#seafloor_age = age_da[:200,:200]
#ny,nx = age_da.shape
#sediment_thickness = np.random.rand(ny, nx) * 5  # Example sediment thickness in meters

params = {
    'H': 50,       # Base rms height in meters
    'kn': 0.05,   # Characteristic width in km^-1 (normal to ridge)
    'ks': 0.5,   # Characteristic length in km^-1 (parallel to ridge)
    'D': 2.2       # Fractal dimension
}


start = time.time()



def process_bathymetry_chunk(coord, chunksize):
    
    print(coord, chunksize)
    
    chunk = age_da[coord[0]:coord[0]+chunksize+chunkpad, coord[1]:coord[1]+chunksize+chunkpad]
    chunk_random = rand_da[coord[0]:coord[0]+chunksize+chunkpad, coord[1]:coord[1]+chunksize+chunkpad]


    if np.all(np.isnan(chunk.data)):
        return chunk
    
    ny,nx = chunk.shape
    
    sediment_thickness = np.random.rand(ny,nx) * 5  # Example sediment thickness in meters

    # Generate the synthetic bathymetry
    synthetic_bathymetry = af.generate_bathymetry_spatial_filter(chunk.data, 
                                                                 sediment_thickness, 
                                                                 params,
                                                                 chunk_random)
    
    return xr.DataArray(synthetic_bathymetry, coords=chunk.coords, name='z')
    

chunksize = 160
chunkpad = 50

coords = np.meshgrid(np.arange(0,1800,chunksize), np.arange(0,3600,chunksize))
coords = list(zip(coords[0].flatten(), coords[1].flatten()))

num_cpus = 6
results = Parallel(n_jobs=num_cpus)(delayed(process_bathymetry_chunk)(coord, chunksize) for coord in coords)
#result = process_bathymetry_chunk(coords[0], chunksize)

print(time.time() - start)



In [ ]:
#xr.combine_by_coords(results)

resss = [result for result in results if 0 not in result.shape]
#resss
#for res in resss:
#    print(res.shape)
#results[24].plot()

#out = xr.combine_by_coords(resss)


fig,ax = plt.subplots(figsize=(20,12))
for res in resss:
    plt.pcolormesh(res.lon, res.lat, res.data)
plt.show()


In [ ]:
sediment_thickness_chunk.plot()
